In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error

In [ ]:
df_train = pd.read_csv('train.csv')
df_train_labels = df_train[['value']]
df_train = df_train.drop(['value'],axis=1)
df_test = pd.read_csv('test.csv')


In [ ]:
df_train.shape, df_test.shape

In [ ]:
df_concatenated = pd.concat([df_train, df_test], ignore_index=True)

In [ ]:
df_concatenated.shape

In [ ]:
df_concatenated['year']=df_concatenated['date'].str[0:4]
df_concatenated['month']=df_concatenated['date'].str[5:7]
df_concatenated['day']=df_concatenated['date'].str[8:]

In [ ]:
df_concatenated = df_concatenated.drop(['date'],axis=1)

In [ ]:
df_concatenated.shape

In [ ]:
df_concatenated.head(2)

In [ ]:
X = df_concatenated
y = df_train_labels

In [ ]:
X.head(2)

In [ ]:
y.head(2)

In [ ]:
X['day']=X['day'].astype('category')
X['month']=X['month'].astype('category')
X['year']=X['year'].astype('category')

In [ ]:
X_onehotencoded = pd.get_dummies(X,['day','month','year'])

In [ ]:
X_train = X_onehotencoded.loc[:10273]

In [ ]:
X_test = X_onehotencoded.loc[10274:]

In [ ]:
y_train = y

In [ ]:
X_train.shape, X_test.shape, y.shape

Random Forest Regressor

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfregr = RandomForestRegressor(max_depth=60, random_state=0, n_estimators=20)
rfregr.fit(X_train, y_train)
scores = cross_val_score(rfregr, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge
ridgeregressor = Ridge(alpha=0.1)
ridgeregressor.fit(X_train,y_train)
scores = cross_val_score(ridgeregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

In [ ]:
y_pred = ridgeregressor.predict(X_test)

In [ ]:
X_test_np = df_test.to_numpy()
f = open('shouviksubmision.csv','a')
f.write('date,value\n')
writevalue=''
for i in range(y_pred.shape[0]):
    writevalue = str(X_test_np[i][0])+','+str(y_pred[i][0])+'\n'
    f.write(writevalue)
    
f.close()    
    

Decision Tree Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtregressor = DecisionTreeRegressor(random_state=0,presort=True)
dtregressor.fit(X_train, y_train)
scores = cross_val_score(dtregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

XgBoost

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.4, learning_rate = 0.5,max_depth = 20, alpha = 7, n_estimators = 10)
xg_reg.fit(X_train, y_train)
scores = cross_val_score(xg_reg, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

In [ ]:
y_pred_xgboost = xg_reg.predict(X_test)

In [ ]:
type(y_pred_xgboost)

In [ ]:
X_test_np = df_test.to_numpy()
f = open('shouviksubmision_2_xgboost.csv','a')
f.write('date,value\n')
writevalue=''
for i in range(y_pred_xgboost.shape[0]):
    writevalue = str(X_test_np[i][0])+','+str(y_pred_xgboost[i])+'\n'
    f.write(writevalue)
    
f.close()    
    

Gradient Boosting

In [ ]:
from sklearn import ensemble

params = {'n_estimators': 540, 'max_depth': 10,'subsample':0.8, 'min_samples_split': 5,'learning_rate': 0.05, 'loss': 'ls'}
gradb = ensemble.GradientBoostingRegressor(**params)

gradb.fit(X_train, y_train)

scores = cross_val_score(gradb, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

In [ ]:
y_pred_gradientboost = gradb.predict(X_test)

X_test_np = df_test.to_numpy()
f = open('shouviksubmision_2_gradientboost.csv','a')
f.write('date,value\n')
writevalue=''
for i in range(y_pred_gradientboost.shape[0]):
    writevalue = str(X_test_np[i][0])+','+str(y_pred_gradientboost[i])+'\n'
    f.write(writevalue)
    
f.close()    
    

In [ ]:
10274/22

Lasso Regression

In [ ]:
from sklearn import linear_model
lassoregressor = linear_model.Lasso(alpha=0.4)
lassoregressor.fit(X_train, y_train)

In [ ]:
scores = cross_val_score(lassoregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(normalize=False).fit(X_train, y_train)

In [ ]:
scores = cross_val_score(linreg, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

CatBoost Regressor

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cboostregressor = CatBoostRegressor()
cboostregressor.fit(X_train, y_train)

grid = {'learning_rate': [0.03, 0.1],'depth': [4, 6, 10],'l2_leaf_reg': [1, 3, 5, 7, 9]}


grid_search_result = cboostregressor.grid_search(grid, X=X_train,y=y_train,plot=True)
# scores = cross_val_score(cboostregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
# (scores.mean()*-1)**0.5


In [ ]:
grid_search_result

In [ ]:
cboostregressor = CatBoostRegressor(depth=10, learning_rate=0.1, l2_leaf_reg=3)
cboostregressor.fit(X_train, y_train)

In [ ]:
scores = cross_val_score(cboostregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

Support Vector Regressor

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV 
# svrregressor = SVR(gamma='scale', C=1.0, epsilon=0.2)
# svrregressor.fit(X_train, y_train)



param_grid = {'kernel': ['sigmoid'],'C': [0.8,0.9,1.0,1.1,1.2],'epsilon': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 0)
grid.fit(X_train, y_train) 


# print(grid.best_params_) 
# print(grid.best_estimator_) 

# scores = cross_val_score(svrregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
# (scores.mean()*-1)**0.5

In [ ]:
print(grid.best_params_) 

rbf kernel

In [ ]:
svrregressor = SVR(kernel='rbf',gamma='scale', C=0.8, epsilon=0.1)
svrregressor.fit(X_train, y_train)


sigmoid kernel

In [ ]:
svrregressor = SVR(kernel='sigmoid',gamma='scale', C=1.2, epsilon=0.8)
svrregressor.fit(X_train, y_train)


linear kernel

In [ ]:
param_grid = {'kernel': ['linear'],'C': [0.8,0.9,1.0,1.1,1.2],'epsilon': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]}

grid = GridSearchCV(SVR(), param_grid, refit = True, verbose = 0)
grid.fit(X_train, y_train) 

In [ ]:
grid.best_params_

In [ ]:
svrregressor = SVR(kernel='linear',gamma='scale', C=1.2, epsilon=0.1) #linear kernel

In [ ]:
svrregressor = SVR(kernel='rbf',gamma='scale', C=1.2, epsilon=0.1) #rbf kernel

In [ ]:
scores = cross_val_score(svrregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(df_train['date'])
plt.show()

AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adbregressor = AdaBoostRegressor(random_state=0, n_estimators=300)
adbregressor.fit(X_train, y_train)
scores = cross_val_score(adbregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

Stochastic Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
params = {'n_estimators': 540, 'subsample': 0.8, 'max_depth': 1, 'min_samples_split': 5,'learning_rate': 0.05, 'loss': 'ls'}
sgradb = ensemble.GradientBoostingRegressor(**params)
sgradb.fit(X_train, y_train)
scores = cross_val_score(sgradb, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

CatBoost without onehot encoding

In [ ]:
from catboost import CatBoostRegressor
X_train_withoutonehotencoding = X.loc[:10273]
X_test_withoutonehotencoding = X.loc[10274:]



In [ ]:
X_train_withoutonehotencoding['year']= X_train_withoutonehotencoding['year'].astype(str)
X_train_withoutonehotencoding['day']= X_train_withoutonehotencoding['day'].astype(str)
X_train_withoutonehotencoding['month']= X_train_withoutonehotencoding['month'].astype(str)




In [ ]:
X_train_withoutonehotencoding.shape, X_test_withoutonehotencoding.shape

In [ ]:
cboostregressor = CatBoostRegressor()
cboostregressor.fit(X_train_withoutonehotencoding, y_train)

grid = {'learning_rate': [0.4,0.5,0.6,0.7],'depth': [4, 6, 10],'l2_leaf_reg': [1, 3, 5, 7, 9]}


grid_search_result = cboostregressor.grid_search(grid, X=X_train_withoutonehotencoding,y=y_train,plot=False)
# scores = cross_val_score(cboostregressor, X_train, y_train, cv=11, scoring='neg_mean_squared_error')
# (scores.mean()*-1)**0.5





In [ ]:
grid_search_result

In [ ]:
cboostregressor = CatBoostRegressor(depth=10, l2_leaf_reg=5, learning_rate=0.5)
cboostregressor.fit(X_train_withoutonehotencoding, y_train)

scores = cross_val_score(cboostregressor, X_train_withoutonehotencoding, y_train, cv=11, scoring='neg_mean_squared_error')
(scores.mean()*-1)**0.5

In [ ]:
depth 10 6.1384

In [ ]:
X_test_withoutonehotencoding['year']= X_test_withoutonehotencoding['year'].astype(str)
X_test_withoutonehotencoding['day']= X_test_withoutonehotencoding['day'].astype(str)
X_test_withoutonehotencoding['month']= X_test_withoutonehotencoding['month'].astype(str)


y_pred_catboost = cboostregressor.predict(X_test_withoutonehotencoding)
y_pred_catboost.shape


In [ ]:
X_test_np = df_test.to_numpy()
f = open('shouviksubmision_2_catboost3.csv','a')
f.write('date,value\n')
writevalue=''
for i in range(y_pred_catboost.shape[0]):
    writevalue = str(X_test_np[i][0])+','+str(y_pred_catboost[i])+'\n'
    f.write(writevalue)
    
f.close()    
    

Deep Learning

In [ ]:
X_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
dlmodel=Sequential()
dlmodel.add(Dense(46, input_shape=(92,), activation='relu'))
# dlmodel.add(Dense(23, input_dim=46, activation='relu'))
dlmodel.add(Dense(1))

In [ ]:
dlmodel.compile(optimizer='adam', loss='mse')
dlmodel.fit(X_train,y_train, epochs=45)

In [ ]:
dlmodel.summary()

In [ ]:
y_pred_dl = dlmodel.predict(X_test)

In [ ]:
X_test_np = df_test.to_numpy()
f = open('shouviksubmision_2_dl.csv','a')
f.write('date,value\n')
writevalue=''
for i in range(y_pred_dl.shape[0]):
    writevalue = str(X_test_np[i][0])+','+str(y_pred_dl[i][0])+'\n'
    f.write(writevalue)
    
f.close()    
    

Plotting the data

In [ ]:
df_train_examine = pd.read_csv('train.csv')

In [ ]:
df_train_examine

In [ ]:
df_train_examine['date']=df_train_examine['date'].astype(str)

In [ ]:
df_train_examine = df_train_examine.set_index('date')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
df_train_examine.plot()
plt.show()